## Trying to programmatically interact with Google Translate

Can I use Selenium to get translations for my tool?

XPath translation error: Message: element not interactable
  (Session info: chrome=137.0.7151.104)
Stacktrace:
0   chromedriver                        0x00000001025e6668 cxxbridge1$str$ptr + 2723108
1   chromedriver                        0x00000001025de8dc cxxbridge1$str$ptr + 2690968
2   chromedriver                        0x0000000102132588 cxxbridge1$string$len + 90032
3   chromedriver                        0x0000000102174404 cxxbridge1$string$len + 359980
4   chromedriver                        0x0000000102173188 cxxbridge1$string$len + 355248
5   chromedriver                        0x000000010216ef94 cxxbridge1$string$len + 338364
6   chromedriver                        0x00000001021bade8 cxxbridge1$string$len + 649232
7   chromedriver                        0x000000010216d9c8 cxxbridge1$string$len + 332784
8   chromedriver                        0x00000001025aa28c cxxbridge1$str$ptr + 2476360
9   chromedriver                        0x00000001025ad520 cxxbridge1$str$ptr + 248930

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time

# Streamlined version based on what's working
def streamlined_translate(language, text):
    """
    Streamlined translation focusing on what works:
    1. Text input (cursor already ready)
    2. Language dropdown click + type + Enter
    3. Translation extraction
    """
    
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    try:
        # Navigate to Google Translate
        driver.get("https://translate.google.com")
        wait = WebDriverWait(driver, 15)
        actions = ActionChains(driver)
        
        # Wait for page to load
        time.sleep(0.5)
        
        # Step 1: Enter text (cursor should be ready in text area)
        print("Entering text...")
        actions.send_keys(text).perform()
        print(f"Entered: {text}")
        
        # Step 2: Click language dropdown and select language
        print("Selecting target language...")
        
        # Find and click the target language button
        target_lang_button = wait.until(EC.element_to_be_clickable((
            By.CSS_SELECTOR, 
            "button[aria-label*='target language'], .VfPpkd-Bz112c-RLmnJb"
        )))
        target_lang_button.click()
        print("Clicked target language dropdown")
        
        # Wait briefly for dropdown, then type language and press Enter
        time.sleep(0.5)
        actions.send_keys(language).perform()
        time.sleep(0.5)
        actions.send_keys(Keys.RETURN).perform()
        print(f"Selected language: {language}")
        
        # Step 3: Wait for translation and extract
        print("Waiting for translation...")
        time.sleep(0.5)
        
        # Extract translation
        translation_selectors = [
            "span[jsname='W297wb']",
            "span[lang]:not([lang='auto']):not([lang=''])",
            ".ryNqvb span",
            ".J0lOec span"
        ]
        
        for selector in translation_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    translated_text = element.text.strip()
                    if translated_text and translated_text != text:
                        print(f"Translation found: {translated_text}")
                        return translated_text
            except:
                continue
        
        print("No translation found")
        return None
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None
        
    finally:
        driver.quit()


In [26]:
# Usage examples:
def test_streamlined():
    # result = streamlined_translate("French", "Parles-tu français?")
    result = streamlined_translate("French", "Parles-tu français?")
    print(f"Result: {result}")

# Uncomment to test
test_streamlined()

# # Usage examples:
# def test_translation():
#     # Test the simple approach first
#     # result = simple_translate("Hello, how are you today?", "Spanish")
#     # print(f"Simple translation result: {result}")
    
#     # Test the full approach
#     result2 = translate_text("French", "Parles-tu français?")
#     print(f"Full translation result: {result2}")

# # Uncomment to test
# test_translation()

Entering text...
Entered: Parles-tu français?
Selecting target language...
Clicked target language dropdown
Selected language: French
Waiting for translation...
Translation found: Do you speak French?
Result: Do you speak French?
